In [1]:
import sys
sys.path.append("../Python")
import ROOT as R
%jsroot on
R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Welcome to JupyROOT 6.29/01
Recompile is not needed


'Utility Functions V1.0.6 \n'

In [2]:
ch_mum = R.TChain("MiniDST")
for i in range(20):
    # print(f"/data/HPS/data/MC/emumu_calchep/events_4p55GeV_{i:03d}_slic_recon.root")
    ch_mum.Add(f"/data/HPS/data/MC/emumu_calchep/events_4p55GeV_dup_{i:03d}_slic_recon.root")
Num_Events = ch_mum.GetEntries()
print(f"Loaded {Num_Events:6.3f} events.")
df = R.RDataFrame(ch_mum)
print("Available data names in Tuple:")
ColumnNames=df.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if str(x).startswith('')]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Loaded 103371.000 events.
Available data names in Tuple:

ecal_cluster_energy           ecal_cluster_hits             ecal_cluster_nhits            ecal_cluster_seed_energy      
ecal_cluster_seed_index       ecal_cluster_seed_ix          ecal_cluster_seed_iy          ecal_cluster_time             
ecal_cluster_uncor_energy     ecal_cluster_uncor_hits       ecal_cluster_uncor_nhits      ecal_cluster_uncor_seed_energy
ecal_cluster_uncor_seed_index ecal_cluster_uncor_seed_ix    ecal_cluster_uncor_seed_iy    ecal_cluster_uncor_time       
ecal_cluster_uncor_x          ecal_cluster_uncor_y          ecal_cluster_uncor_z          ecal_cluster_x                
ecal_cluster_y                ecal_cluster_z                ecal_hit_energy               ecal_hit_index_x              
ecal_hit_index_y              ecal_hit_time                 ecal_hit_x                    ecal_hit_y                    
ecal_hit_z                    event_number                  ext_trigger                   hodo_

In [3]:
df = df.Define("fiducial_cut","auto f=fiducial_cut(ecal_cluster_seed_ix,ecal_cluster_seed_iy); return f;")
h_ecal = df.Histo2D(("h_ecal","Ecal Seed Hits;ix;iy",50,-25.5,24.5,13,-6.5,6.5), "ecal_cluster_seed_ix", "ecal_cluster_seed_iy")
df_fcut = df.Filter("for(auto fid: fiducial_cut){ if(!fid){return(false);}}; return true;")
h_ecal_fid = df_fcut.Histo2D(("h_ecal_fid","Ecal Seed Hits F-Cut;ix;iy",50,-25.5,24.5,13,-6.5,6.5), "ecal_cluster_seed_ix", "ecal_cluster_seed_iy")

h_pdg = df.Histo1D(("h_pdg","PDG Id",41,-20.5,20.5),"mc_part_pdg")

In [4]:
cc0 = R.TCanvas("cc0","CC0",800, 1000)
cc0.Divide(1,2)
p1 = cc0.cd(1)
h_ecal.Draw("colz")
p2 = cc0.cd(2)
h_ecal_fid.Draw("colz")
cc0.Draw()

In [12]:
dfx = df_fcut\
    .Define("one_crystal_e","RVec<double> out; for(int i=0; i< ecal_cluster_nhits.size(); ++i){ if(ecal_cluster_nhits[i]==1){ out.push_back(ecal_cluster_energy[i]);} } return out;") \
    .Define("one_crystal_se","RVec<double> out; for(int i=0; i< ecal_cluster_nhits.size(); ++i){ if(ecal_cluster_nhits[i]==1){ out.push_back(ecal_cluster_seed_energy[i]);} } return out;") \
    .Define("one_crystal_ue","RVec<double> out; for(int i=0; i< ecal_cluster_nhits.size(); ++i){ if(ecal_cluster_nhits[i]==1){ out.push_back(ecal_cluster_uncor_energy[i]);} } return out;")
h_1c_e = dfx.Histo1D(("h_1c_e","One crystal energy;E [GeV]",200,0.,1.),"one_crystal_e")
h_1c_se = dfx.Histo1D(("h_1c_se","One crystal seed energy; E [GeV]",200,0.,1.),"one_crystal_se")
h_1c_ue = dfx.Histo1D(("h_1c_ue","One crystal seed energy; E [GeV]",200,0.,1.),"one_crystal_ue")


In [13]:
cc0 = R.TCanvas("cc0","CC0",800, 800)
cc0.Divide(1,1)
p1 = cc0.cd(1)
h_1c_se.SetStats(0)
h_1c_se.SetLineColor(R.kRed)
h_1c_se.SetLineWidth(2)
h_1c_se.Draw()
h_1c_e.Draw("same")
h_1c_ue.SetLineColor(R.kGreen)
h_1c_ue.Draw("same")

cc0.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0
